## Korpuserstellung 

Dieser Code liest vollständig die Datei `thunberg_reisen02_1794.tcf.xml` aus und schreibt den original, sowie den standardisierten Text in eine json-Datei.

Anschließend werden die ausgezeichneten Personen- und Ortsnamen aus `Thunberg_Reisen_2_1794.xml` ausgelesen und in die `reisen02_tcf.json` integriert.

In [1]:
import xml.etree.ElementTree as ET
import json
import re

# ---------- Schritt 1: Entity-IDs aus zweiter XML lesen ----------
entity_ids = {}
pers_count = 0
place_count = 0

tree_entities = ET.parse('Thunberg_Reisen_2_1794.xml')
root_entities = tree_entities.getroot()

ns_tei = {'tei': 'http://www.tei-c.org/ns/1.0'}  # TEI-XML Namespace

# <persName> Elemente
for pers in root_entities.findall('.//tei:persName', namespaces=ns_tei):
    for w in pers.findall('.//tei:w', namespaces=ns_tei):
        wid = w.get('{http://www.w3.org/XML/1998/namespace}id')
        if wid:
            entity_ids[wid] = 'persName'
            pers_count += 1

# <placeName> Elemente
for place in root_entities.findall('.//tei:placeName', namespaces=ns_tei):
    for w in place.findall('.//tei:w', namespaces=ns_tei):
        wid = w.get('{http://www.w3.org/XML/1998/namespace}id')
        if wid:
            entity_ids[wid] = 'placeName'
            place_count += 1

# ---------- Schritt 2: TCF-Datei einlesen und verarbeiten ----------
tree = ET.parse('thunberg_reisen02_1794.tcf.xml')
root = tree.getroot()

ns = {'tc': 'http://www.dspin.de/data/textcorpus'}

tokens_elem = root.find('.//tc:tokens', namespaces=ns)
sentences_elem = root.find('.//tc:sentences', namespaces=ns)
orthography_elem = root.find('.//tc:orthography', namespaces=ns)

token_map = {}
global_index = 0
for token in tokens_elem.findall('tc:token', namespaces=ns):
    token_id = token.get('ID')
    token_text = token.text
    token_map[token_id] = {
        'text': token_text,
        'global_token_index': global_index
    }
    global_index += 1

corrections_map = {}
if orthography_elem is not None:
    for correction in orthography_elem.findall('tc:correction', namespaces=ns):
        corr_text = correction.text
        corr_token_ids = correction.get('tokenIDs', '').split()
        for tid in corr_token_ids:
            corrections_map[tid] = corr_text

# ---------- Schritt 3: Sätze mit XML-Token-ID und Entitäten versehen ----------
sentences_json = []
matched_pers = 0
matched_place = 0

for i, sentence in enumerate(sentences_elem.findall('tc:sentence', namespaces=ns)):
    token_ids = sentence.get('tokenIDs').split()
    orig_tokens = []
    norm_tokens = []

    for local_id, tid in enumerate(token_ids):
        token_info = token_map.get(tid)
        if not token_info:
            continue
        global_idx = token_info['global_token_index']
        original_text = token_info['text']
        corrected_text = corrections_map.get(tid, original_text)

        xml_token_id = tid  # ← die xml:id, mit der verglichen wird

        entity_type = entity_ids.get(xml_token_id)

        if entity_type == 'persName':
            matched_pers += 1
        elif entity_type == 'placeName':
            matched_place += 1

        orig_token = {
            'token_id': local_id,
            'xml_token_id': xml_token_id,
            'text': original_text,
            'global_token_index': global_idx
        }
        norm_token = {
            'token_id': local_id,
            'xml_token_id': xml_token_id,
            'text': corrected_text,
            'global_token_index': global_idx
        }

        if entity_type:
            orig_token['entity_type'] = entity_type
            norm_token['entity_type'] = entity_type

        orig_tokens.append(orig_token)
        norm_tokens.append(norm_token)

    def clean_spacing(text):
        return re.sub(r'\s+([.,;:!?])', r'\1', text)

    orig_sentence_text = clean_spacing(" ".join(tok["text"] for tok in orig_tokens))
    norm_sentence_text = clean_spacing(" ".join(tok["text"] for tok in norm_tokens))

    sentences_json.append({
        'sentence_id': i,
        'orig_sentence': {
            'text': orig_sentence_text,
            'tokens': orig_tokens
        },
        'norm_sentence': {
            'text': norm_sentence_text,
            'tokens': norm_tokens
        }
    })

# ---------- Schritt 4: JSON schreiben ----------
with open('reisen02_tcf.json', 'w', encoding='utf-8') as f:
    json.dump({'sentences': sentences_json}, f, ensure_ascii=False, indent=2)

# ---------- Schritt 5: Ergebnis ausgeben ----------
print("Fertig! JSON gespeichert als 'reisen02_tcf.json'.")
print(f"Entitäten aus zweiter XML: {pers_count} persName, {place_count} placeName")
print(f"Tatsächlich übertragene Entitäten: {matched_pers} persName, {matched_place} placeName")

# ---------- Schritt 6: Fehlende Entity-IDs ausgeben ----------

# Set aller xml_token_ids, die im JSON vorkommen
json_token_ids = set()

for sentence in sentences_json:
    for token in sentence['orig_sentence']['tokens']:
        if 'xml_token_id' in token:
            json_token_ids.add(token['xml_token_id'])

# Fehlende IDs aus placeName
missing_place_ids = [tid for tid, typ in entity_ids.items()
                     if typ == 'placeName' and tid not in json_token_ids]

# Ausgabe
if missing_place_ids:
    print(f"\nEs wurden {len(missing_place_ids)} placeName-IDs nicht übertragen:")
    for mid in missing_place_ids:
        print(f"  - {mid}'")
else:
    print("\n✅ Alle placeName-IDs wurden erfolgreich übertragen.")




Fertig! JSON gespeichert als 'reisen02_tcf.json'.
Entitäten aus zweiter XML: 422 persName, 1677 placeName
Tatsächlich übertragene Entitäten: 422 persName, 1672 placeName

Es wurden 5 placeName-IDs nicht übertragen:
  - w47b8_1'
  - w7c71_1'
  - wb717_1'
  - w10d79_1'
  - w14915_1'


Vier der fünf fehlenden Ortsnamen sind nur scheinbar nicht vorhanden, da ihre IDs mit "_" auf eine Worttrennung verweisen. In der JSON sind die vollständigen Wörter aber letztendlich dennoch korrekt ausgezeichnet worden. Nur die die erste ID wurd aufrund eienr fehlerhaften Auszeichnung nicht korrekt übertragen, was an dieser Stelle aber nicht weiter stören soll.

Das nachstehende Skript prüft noch einmal die Korrektheit der übertragennen Begriffe. Zunächst scheinen etliche Namen Fehlerhaft zu sein, beim näheren Hinsehen handelt es sich aber nur um Normalisierungs-Unterschiede oder weitere Worttrennungen. Letztlich sind in der JSON die richtigen Namen ausgezeichnet.






In [2]:
import xml.etree.ElementTree as ET
import json

# ---------- XML einlesen ----------
NS = {'tei': 'http://www.tei-c.org/ns/1.0'}
tree = ET.parse("Thunberg_Reisen_2_1794.xml")
root = tree.getroot()

# Alle <w> sammeln, die Teil von persName oder placeName sind
xml_tokens = {}

for tag in ["persName", "placeName"]:
    for ent in root.findall(f".//tei:{tag}", namespaces=NS):
        for w in ent.findall(".//tei:w", namespaces=NS):
            wid = w.get("{http://www.w3.org/XML/1998/namespace}id")
            text = w.text.strip() if w.text else ""
            if wid:
                xml_tokens[wid] = text

# ---------- JSON einlesen ----------
with open("reisen02_tcf.json", encoding="utf-8") as f:
    json_data = json.load(f)

json_tokens = {}

for sentence in json_data["sentences"]:
    for token in sentence["norm_sentence"]["tokens"]:
        wid = token.get("xml_token_id")
        if wid:
            json_tokens[wid] = token["text"]

# ---------- Vergleich ----------
mismatches = []

for wid, xml_text in xml_tokens.items():
    json_text = json_tokens.get(wid)
    if json_text is not None:
        if xml_text != json_text:
            mismatches.append((wid, xml_text, json_text))

# ---------- Ausgabe ----------
print(f"Vergleich abgeschlossen.")
print(f"Verglichene IDs: {len(xml_tokens)}")
print(f"Übereinstimmend: {len(xml_tokens) - len(mismatches)}")
print(f"Nicht übereinstimmend: {len(mismatches)}\n")

if mismatches:
    print("Nicht übereinstimmende Tokens:")
    for wid, xml_text, json_text in mismatches:
        print(f"  - {wid}: XML='{xml_text}' vs. JSON='{json_text}'")



Vergleich abgeschlossen.
Verglichene IDs: 2099
Übereinstimmend: 1469
Nicht übereinstimmend: 630

Nicht übereinstimmende Tokens:
  - w3d: XML='Chriſtian' vs. JSON='Christian'
  - w406: XML='Rudenſchoͤld' vs. JSON='Rudenschöld'
  - w41b: XML='Baͤck' vs. JSON='Bäck'
  - wec8: XML='Linné' vs. JSON='Linne'
  - wed2: XML='Lidrén' vs. JSON='Lidren'
  - wf12: XML='Hornſtedt' vs. JSON='Hornstedt'
  - wf1f: XML='Caßſtröm' vs. JSON='Caßström'
  - wf74: XML='Engeſtröm' vs. JSON='Engeström'
  - wfa5: XML='Borgſtröm' vs. JSON='Borgström'
  - wfad: XML='Heſſelius' vs. JSON='Hesselius'
  - wfb5: XML='Berg' vs. JSON='Berg.'
  - w1002: XML='Gallén' vs. JSON='Gallen'
  - w1019: XML='Schalén' vs. JSON='Schalen'
  - w104b: XML='Haſt' vs. JSON='Hast'
  - w1082: XML='Ahl' vs. JSON='Ahl.'
  - w10d1: XML='Carlſon' vs. JSON='Carlson'
  - w10e8: XML='Åckermann' vs. JSON='Ackermann'
  - w94a: XML='Kaͤmpfers' vs. JSON='Kämpfers'
  - wa82: XML='Kaͤmpfers' vs. JSON='Kämpfers'
  - w1dba: XML='Kaͤmpfer' vs. JSON='Kämp

In ersten Versuchen zeigte sich schnell, dass sich in dem Texte zwei größere Abschnitte befinden, in denen japanische Übersetzungen vermerkt sind.

Zusätzlich findet sich einer kürzerer Abschnitt mit japanischen Zahlwörtern und am Ende des Bandes wurde ein Korrekturliste von lateinsichen Pflanzennamen aus dem ersten Band angehängt.

Diese Abschnitte werden unweigerlich zu Problemen führen und deshalb kurzerhand entfernt. Sie lassen sich im restlichen Textlayout auch leicht identifizieren, wie nachstehende Skriptausgabe zeigen soll.

Im Detail handelt es sich um folgende Abschnitte:

Satz 3090 - 5012

Satz 6508 - 7071

Satz 8320 - 8350

Satz 8576 - 8611

In [3]:
import json

# Datei laden
input_file = "reisen02_tcf.json"

# JSON-Datei laden
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Alle sentence_ids und zugehörigen Texte ausgeben
for sentence in data["sentences"]:
    sentence_id = sentence["sentence_id"]
    norm_text = sentence["norm_sentence"]["text"]
    print(f"Sentence ID: {sentence_id}, Text: {norm_text}")

Sentence ID: 0, Text: Karl Peter Thunbergs, Ritters des Königlichen Schwedischen Wasaordens, Doktors der Arzneigelahrtheit, Professors der Botanik zu Upsala, und Mitgliedes verschiedener, einheimischer und auswärtiger Akademien und gelehrter Gesellschaften, Reise durch einen Teil von Europa, Afrika und Asien, hauptsächlich in Japan, in den Jahren 1770 bis 1779.
Sentence ID: 1, Text: Aus dem Schwedischen frei übersetzt von Christian Heinrich Groskurd, Rektor des Gymnasiums zu Stralsund.
Sentence ID: 2, Text: Mit Kupfern.
Sentence ID: 3, Text: Zweiter Band.
Sentence ID: 4, Text: Berlin bei Haude und Spener, 1794.
Sentence ID: 5, Text: Vorrede des Verfassers.
Sentence ID: 6, Text: Dieser zweite Band meiner Reisebeschreibung enthält die Reise von Batavia nach Japan, den Aufenthalt auf der Insel Dezime, die Reise nach Jedo, der Hauptstadt von Japan, den Aufenthalt daselbst, die Rückreise nach Dezime, mannigfaltige Nachrichten von diesem Lande und dessen Einwohnern; Ferner meine Rückreise na

Folgender Code-Block entfernt die entsprechenden Abschnitte:
3090 bis 5012
6508 bis 7071
8320 bis 8350
8576 bis 8611 

In [4]:
import json

# Datei laden
input_file = "reisen02_tcf.json"
output_file = "reisen02_filtered.json"

# ID-Ranges angeben
id_ranges = [
    range(3090, 5013),
    range(6508, 7072),
    range(8320, 8351),
    range(8576, 8612)
]

# JSON-Datei laden
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Sätze filtern
filtered_sentences = []
for sentence in data["sentences"]:
    sentence_id = sentence["sentence_id"]
    # Prüfen, ob die sentence_id in einem der Ranges liegt
    if not any(sentence_id in id_range for id_range in id_ranges):
        filtered_sentences.append(sentence)

# Gefilterte Daten speichern
data["sentences"] = filtered_sentences
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Die Sätze in den angegebenen Bereichen wurden entfernt. Gefilterte Datei: {output_file}")

Die Sätze in den angegebenen Bereichen wurden entfernt. Gefilterte Datei: reisen02_filtered.json


Die nun vorhandene `reisen02_filtered.json` kann für die Textnormalisierung und Informationsextraktion mit gemma3 genutzt werden.

In [ ]:
import json

# Datei laden
input_file = "reisen02_filtered.json"

# JSON-Datei laden
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Alle sentence_ids und zugehörigen Texte ausgeben
for sentence in data["sentences"]:
    sentence_id = sentence["sentence_id"]
    norm_text = sentence["norm_sentence"]["text"]
    print(f"Sentence ID: {sentence_id}, Text: {norm_text}")